First we download, compile, and install LAMMPS 

In [ ]:
# !sudo apt install cmake build-essential git python3.10-venv

In [ ]:
# !pip3 install nglview
# !pip3 install ase
# !pip3 install matplotlib
# !pip3 install mdanalysis
# !pip3 install tidynamics

In [ ]:
# from google.colab import output
# output.enable_custom_widget_manager()

In [ ]:
# !rm -rf lammps
# !git clone -b stable --depth=1 https://github.com/lammps/lammps.git lammps

In [ ]:
# !rm -rf build
# !mkdir build
# !cd build; cmake ./../lammps/cmake -D CMAKE_INSTALL_PREFIX=/usr -D BUILD_SHARED_LIBS=on -D LAMMPS_EXCEPTIONS=on -DPYTHON_EXECUTABLE=$(which python3) -D PKG_PYTHON=on -D BUILD_LIB=yes -D BUILD_SHARED_LIBS=yes -D PKG_KSPACE=YES -D PKG_MISC=YES -D PKG_RIGID=YES -D PKG_MOLECULE=YES -D PKG_DIELECTRIC=YES -D PKG_DIPOLE=YES -D PKG_EXTRA-PAIR=YES ; make -j 6

In [ ]:
# !cd build; make install-python

Now we start the LAMMPS python interface

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt #import required modules

In [ ]:
from lammps import PyLammps #start lammps python interface
L = PyLammps()

In [ ]:
L.clear() #restart lammps

In [ ]:
dt = 0.001 #timestep in ps
numsteps = 100000 # number of timesteps to run for our calculation
camass = 40.078 #mass of calcium in au
fmass = 18.998 #mass of fluorine in au
cacharge = 2 #mass of calcium in au
fcharge = -1 #mass of fluorine in au
temperature = 3000 #temperature in kelvin
inittemp = temperature
starttemp = temperature
endtemp = temperature
tDamp = 0.1 #temperature damping time in ps (100x timestep is a good starting point
inputfile = 'caf2.lmpdat' #input file with coordinates

In [ ]:
########potential parameters#########
cutoff = 10 #cutoff distance of LJ potential
###First for Ca-Ca
cacae = 0.08 # 0.02
cacasig = 4.10 #3.82
###Then for Ca-F
cafe = 0.03
cafsig = 2.4

In [ ]:
L.clear() #restart lammps

In [ ]:
L.units('metal') #choose which units > mass = grams/mole / distance = Angstroms / time = picoseconds / energy = eV / temperature = Kelvin
L.dimension('3') #number of dimensions for simulation
L.boundary('p', 'p', 'p') # periodicity of your unit-cell p = periodic
L.atom_style('charge') # choose atom type

In [ ]:
L.read_data(inputfile) # read input file

In [ ]:
L.replicate('1', '1', '1') # replicate once in each direction (no change)
L.mass('1', camass) # set mass of atom 1
L.mass('2', fmass) # set mass of atom 2
L.set('type', '1', 'charge', cacharge) #set charge of atom 1
L.set('type', '2', 'charge', fcharge) #set charge of atom 2
L.group('ca', 'type', '1') #set atom 1 to be called 'ca'
L.group('f', 'type', '2') #set atom 2 to be called 'f'

In [ ]:
L.neighbor('3.0', 'bin') #build a neighbour list
L.neigh_modify('every', '1', 'delay', '0', 'check', 'yes', 'one', '1000') #update the neighbour list every timestep
L.pair_style('lj/cut/coul/long', cutoff) # state the potential to be LJ with coulombic forces
L.kspace_style('pppm', '1.0e-4')  # how to sample periodicity of coulomb forces

In [ ]:
L.pair_coeff('1', '1', cacae, cacasig, cutoff) #set pairwise potential
L.pair_coeff('1', '2', cafe, cafsig, cutoff) #set pairwise potential
L.pair_coeff('2', '2', '0', '1', cutoff) #set pairwise potential

In [ ]:
L.timestep(dt) #set the timestep
L.velocity('all', 'create', inittemp, random.randrange(1000)) #set initial velocities (instantaneous temperature) with RNG

In [ ]:
L.variable('varVolume', 'equal', 'vol') #set volume to a variable that can be printed out
L.thermo_style('custom', 'step', 'time', 'temp', 'pe', 'etotal', 'enthalpy', 'press', 'vol', 'lx', 'ly', 'lz') #choose print out parameters
L.thermo_modify('format', '4', '%20.15g') #choose format (sig figs) of print outs

Now we do our first calculation. We optimise the unitcell to its most stable structure

In [ ]:
L.fix('1', 'all', 'box/relax', 'iso', '1.0', 'vmax', '0.001') #allow unit cell to relax
L.min_style('cg') #choose algorithm for optimisation (conjugate gradient)
L.minimize('1e-20', '1e-20', '1000', '1000') #run optimisation
L.unfix('1') #remove fix - don't need anymore

In [ ]:
L.write_data('structure.data') #write out optimised structure to a file

In [ ]:
L.variable('celllength', 'equal', 'lx') #make new variable to print out

In [ ]:
print('unit cell length =', L.variables['celllength'].value, 'A and should be close to experimental 16.548155 A') #check with experimental unit cel to see if our potential is reasonable

In [ ]:
L.fix('1', 'all', 'nvt', 'temp', starttemp, endtemp, tDamp) #set the system under the nvt ensemble at our designated temperature
L.fix('2', 'all', 'momentum', '100', 'linear', '1 1 1') #don't allow the system to just move all together (not physical)
L.run(10000) #equilibriate the system

In [ ]:
######Can calculate MSD within lammps but it is mega bugged#######
# L.variable('msd4all', 'equal', 'c_allMSD[4]')
# L.variable('msd4f', 'equal', 'c_fMSD[4]')
# L.variable('msd4ca', 'equal', 'c_caMSD[4]')

In [ ]:
L.dump('traj_xyz all custom 100 caf2.lammpsdump id type mass xu yu zu fx fy fz vx vy vz') #write out trajectory to a file
L.dump_modify('traj_xyz element F Ca sort id') #sort file by element

In [ ]:
L.run(numsteps) #run our calculation

In [ ]:
#######would use if not bugged###########
# print('msd for f=', L.variables['msd4f'].value, 'A^2/ps')
# print('msd for ca=', L.variables['msd4ca'].value, 'A^2/ps')

Now we perform analysis to find the MSD (Mean Squared Displacement) of our ions.

In [ ]:
#####import tools#####
import MDAnalysis as mda
import MDAnalysis.analysis.msd as msd

In [ ]:
u = mda.Universe('caf2.lammpsdump', atom_style="id type x y z fx fy fz vx vy vz", lengthunit="a", timeunit="fs")
MSDF = msd.EinsteinMSD(u, select='type 2', msd_type='xyz', fft=True) #type 1 is ca 2 is f
MSDCa = msd.EinsteinMSD(u, select='type 1', msd_type='xyz', fft=True) #type 1 is ca 2 is f
MSDF.run()
MSDCa.run()

Now we plot out our results

In [ ]:
msdf =  MSDF.results.timeseries
msdca = MSDCa.results.timeseries
nframesf = MSDF.n_frames
nframesca = MSDCa.n_frames
timestep = 100  # this needs to be the actual time between frames
lagtimesf = np.arange(nframesf) * timestep  # make the lag-time axis
lagtimesca = np.arange(nframesca) * timestep  # make the lag-time axis

fig = plt.figure()
ax = plt.axes()
# plot the actual MSD
ax.plot(lagtimesf, msdf, ls="-", label=r'F')
exactf = lagtimesca * 6

ax.plot(lagtimesca, msdca, ls="-", label=r'Ca')
exactca = lagtimesca * 6

# plot the exact result
plt.xlabel('$Time (fs)$')
plt.ylabel('$MSD (\AA^2)$')

slopef, interceptf = np.polyfit((exactf), (msdf), 1)
slopeca, interceptca = np.polyfit((exactca), (msdca), 1)
slopecorrectedf = slopef * 1000
slopecorrectedca = slopeca * 1000

plt.text(50, 75, ("Diffusion coefficient F = " + str(round(slopecorrectedf, 4)) + " × $10^{-5}$ cm$^2$/s"))
plt.text(50, 400, ("Diffusion coefficient Ca = " + str(round(slopecorrectedca, 4)) + " × $10^{-5}$ cm$^2$/s"))

print("Diffusion coefficient F = " + str(slopecorrectedf) + " cm^2/s")
print("Diffusion coefficient Ca = " + str(slopecorrectedca) + " cm^2/s")

Now we can view our trajectory as an animation

In [ ]:
import nglview as nv
from ase.io import read

In [ ]:
atoms = read('caf2.lammpsdump', index=':', specorder=[7, 20])

In [ ]:
view = nv.show_asetraj(atoms)

In [ ]:
view.clear_representations()
view.add_unitcell(selections='all')
view.add_spacefill(selections='all')
view.update_representation()

In [ ]:
view

1. Consider the system at a series of temperatures (300K, 500K, 1000K, 1500K, 2000K, 2500K, 3000K). Plot the diffusion coefficients of the ions as a function of temperature. Comment on how the diffusion coefficient changes for the two different ions at various temperatures.
2. Read the lammps documentation and change the ensemble from NVT to NPT. Calculate the diffusion coefficient at 2000K. Comment on the difference of the diffusion coefficient for the NVT vs the NPT ensemble. If the diffusion coefficient of a material is wanted for 'real world' applications, which ensemble should be used and why? Suggest a scenario where the NVT ensemble describes the system of study well.